In [1]:
import logging
import os
import argparse
import math
import numpy as np

import traceback

from tqdm import tqdm

import pandas as pd

from src.config.config import Config, load_config_from_json
from src.evaluate.rouge_evaluator import ScoreSummary
from src.loader.class_loader import Dataset, SOURCE, Cluster
from src.utils.factory import create_model, create_evaluator
from src.model.sds.combination import CombinationRanker
from src.loader.class_loader import load_cluster

In [2]:
config = load_config_from_json()

try:
    train_set = load_cluster(
        config.train_path,
    )
    logging.warning("[PIPELINE] - Load train set from {}. Done.".format(config.train_path))
except Exception as e:
    train_set = None
    logging.warning("[PIPELINE] - Load train set from {}. Failed. Using None.".format(config.train_path))
    print(e)

try:
    valid_set = load_cluster(
        config.valid_path,
    )
    logging.warning("[PIPELINE] - Load valid set from {}. Done.".format(config.valid_path))
except Exception as e:
    valid_set = None
    logging.warning("[PIPELINE] - Load valid set from {}. Failed. Using None.".format(config.valid_path))
    print(e)

try:
    test_set = load_cluster(
        "/home/dang/vlsp-final-year/dataset/vlsp_abmusu_test_data.jsonl",
    )
    logging.warning("[PIPELINE] - Load test set from {}. Done.".format("/home/dang/vlsp-final-year/dataset/vlsp_abmusu_test_data.jsonl"))
except Exception as e:
    test_set = None
    logging.warning("[PIPELINE] - Load test set from {}. Failed. Using None.".format("/home/dang/vlsp-final-year/dataset/vlsp_abmusu_test_data.jsonl"))
    print(e)

Total number of cluster:  200


200it [00:38,  5.23it/s]


Total number of cluster:  100


100it [00:18,  5.36it/s]


Total number of cluster:  300


300it [00:49,  6.09it/s]


In [3]:
train_set.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
valid_set.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
test_set.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)

train_scores = []
valid_scores = []
test_scores = []

In [4]:
from src.model.sds.combination import CombinationRanker

config = load_config_from_json()
model_config = config.models[15]

model = CombinationRanker(model_config)
model.training(valid_set)

Loading codes from /home/dang/vlsp-final-year/external/sentence_transformer/vn_sbert_deploy/bpe/bpe.codes ...
Read 64000 codes from the codes file.


In [5]:
weight = {
    "tfidf": 0.1,
    "lexrank": 0.1,
    "textrank": 0.1
}
    
print("get local score on train set")
for cluster in tqdm(train_set.clusters):
    document_score = []

    for doc in cluster.documents:
        document_score.append(model.get_score(doc.get_all_sents(), 1))

    train_scores.append(document_score)

print("get local score on valid set")
for cluster in tqdm(valid_set.clusters):
    document_score = []

    for doc in cluster.documents:
        document_score.append(model.get_score(doc.get_all_sents(), 1))

    valid_scores.append(document_score)
    
print("get local score on test set")
for cluster in tqdm(test_set.clusters):
    document_score = []

    for doc in cluster.documents:
        document_score.append(model.get_score(doc.get_all_sents(), 1))

    test_scores.append(document_score)

get local score on train set


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:43<00:00,  4.64it/s]


get local score on valid set


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00,  4.88it/s]


get local score on test set


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:59<00:00,  5.06it/s]


In [6]:
from src.evaluate.rouge_evaluator import RougeScore

class RougeScoreStorage:
    def __init__(self):
        self.df = pd.DataFrame(columns=[
            'cluster_id',
            'rouge_1_p',
            'rouge_1_r',
            'rouge_1_f',
            'rouge_2_p',
            'rouge_2_r',
            'rouge_2_f',
            'rouge_l_p',
            'rouge_l_r',
            'rouge_l_f',
        ])
    
    def add_score(self, cluster_id: int, score: RougeScore):
        self.df = self.df.append({
            'cluster_id': cluster_id,
            'rouge_1_p': score.rouge1.p,
            'rouge_1_r': score.rouge1.r,
            'rouge_1_f': score.rouge1.f1,
            'rouge_2_p': score.rouge2.p,
            'rouge_2_r': score.rouge2.r,
            'rouge_2_f': score.rouge2.f1,
            'rouge_l_p': score.rougeL.p,
            'rouge_l_r': score.rougeL.r,
            'rouge_l_f': score.rougeL.f1,
        }, ignore_index=True)
        
    def summary_score(self):
        summary_df = pd.DataFrame(columns=[
            'name',
            'mean',
            'min',
            'max',
            'std',
        ])

        metric_cols = [
            'rouge_1_p',
            'rouge_1_r',
            'rouge_1_f',
            'rouge_2_p',
            'rouge_2_r',
            'rouge_2_f',
            'rouge_l_p',
            'rouge_l_r',
            'rouge_l_f', ]

        for col in metric_cols:
            describe = self.df[col].describe()
            summary_df = summary_df.append({
                'name': col,
                'mean': describe['mean'],
                'min': describe['min'],
                'max': describe['max'],
                'std': describe['std'],
            }, ignore_index=True)
        
        return summary_df

In [7]:
from src.evaluate.rouge_evaluator import ScoreSummary
from src.utils.factory import create_model, create_evaluator
from src.model.mmr_query import MMRSummarizerQuery

evaluator = create_evaluator(config.eval_config)

mmr = MMRSummarizerQuery(config.models[15])

Loading codes from /home/dang/vlsp-final-year/external/sentence_transformer/vn_sbert_deploy/bpe/bpe.codes ...
Read 64000 codes from the codes file.


In [8]:
n_sent = [11, 0.2]

def get_rouge_score(weights):
    train_storage = RougeScoreStorage()
    valid_storage = RougeScoreStorage()
    
    print("Test on training set")
    for idx, cluster in tqdm(enumerate(train_set.clusters)):
        chosen_sent = [] 
        
        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)
        
        
        for idxc, doc in enumerate(cluster.documents):
            sents = doc.get_all_sents()
            
            scores = train_scores[idx][idxc]
            
            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)
        
            for key in weights.keys():
                combine_score += scores[key] * weights[key]
            
            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]
            
            for i in chosen_idx:
                chosen_sent.append(sents[i]) 
                
        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())
                
        train_storage.add_score(
            cluster.cluster_idx,
            evaluator(
                '.'.join(pred_sent),
                '.'.join(cluster.get_summary()),
            )
        )
    
    print("Test on valid set")
    for idx, cluster in tqdm(enumerate(valid_set.clusters)):
        chosen_sent = [] 
        
        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)
        
        
        for idxc, doc in enumerate(cluster.documents):
            sents = doc.get_all_sents()
            
            scores = valid_scores[idx][idxc]
            
            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)
        
            for key in weights.keys():
                combine_score += scores[key] * weights[key]
            
            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]
            
            for i in chosen_idx:
                chosen_sent.append(sents[i]) 
                
        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())
                
        valid_storage.add_score(
            cluster.cluster_idx,
            evaluator(
                '.'.join(pred_sent),
                '.'.join(cluster.get_summary()),
            )
        )
        
    print("Using weight\n", weights)
    print("Train result\n", train_storage.summary_score())
    print("Valid result\n", valid_storage.summary_score())

In [9]:
def get_rouge_score_and_saved(weights):
    train_storage = RougeScoreStorage()
    valid_storage = RougeScoreStorage()
    
    print("Test on training set")
    
    """train_set.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
    for idx, cluster in tqdm(enumerate(train_set.clusters)):
        chosen_sent = [] 

        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)


        for idxc, doc in enumerate(cluster.documents):
            doc.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
            sents = doc.get_all_sents()

            scores = train_scores[idx][idxc]

            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)

            for key in weights.keys():
                combine_score += scores[key] * weights[key]

            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]

            for i in chosen_idx:
                chosen_sent.append(sents[i]) 

        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())

        train_storage.add_score(
            cluster.cluster_idx,
            evaluator(
                '.'.join(pred_sent),
                '.'.join(cluster.get_summary()),
            )
        )"""
    
    print("Test on valid set")
    
    valid_set.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
    for idx, cluster in tqdm(enumerate(valid_set.clusters)):
        chosen_sent = [] 
        
        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)
        
        
        for idxc, doc in enumerate(cluster.documents):
            doc.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
            sents = doc.get_all_sents()
            
            scores = valid_scores[idx][idxc]
            
            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)
        
            for key in weights.keys():
                combine_score += scores[key] * weights[key]
            
            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]
            
            for i in chosen_idx:
                chosen_sent.append(sents[i]) 
                
        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())
        
        cluster.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
        valid_storage.add_score(
            cluster.cluster_idx,
            evaluator(
                '.'.join(pred_sent),
                '.'.join(cluster.get_summary()),
            )
        )
        if idx == 0:
            print("Predict:\n", '.'.join(pred_sent))
            print("Golden:\n", '.'.join(cluster.get_summary()))
     
    print("Predict on test set")
    
    test_set.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
    predictions = []
    
    for idx, cluster in tqdm(enumerate(test_set.clusters)):
        chosen_sent = [] 
        
        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)
        
        
        for idxc, doc in enumerate(cluster.documents):
            doc.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
            sents = doc.get_all_sents()
            
            scores = test_scores[idx][idxc]
            
            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)
        
            for key in weights.keys():
                combine_score += scores[key] * weights[key]
            
            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]
            
            for i in chosen_idx:
                chosen_sent.append(sents[i]) 
                   
        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())
        predictions.append(' '.join(pred_sent))
        
        if idx == 0:
            print("Predict:\n", ' '.join(pred_sent))
        
    print("Start write to txt")
    with open(os.path.join("/home/dang/vlsp-final-year/data/result/outer/combination", "results.txt"), "w") as f:
        for summary in predictions:
            summary.replace('_', ' ')
            f.write(summary)
            f.write('\n')
    print("Done write to txt")
                
        
        
    print("Using weight\n", weights)
    #print("Train result\n", train_storage.summary_score())
    print("Valid result\n", valid_storage.summary_score())

In [10]:
# for i in [idx * .2 for idx in range(5)]:
#     for j in [idx * .2 for idx in range(5)]:
#         if i + j > 1.0:
#             continue 
#         k = 1.0 - i - j 
        
#         weight = {
#             "tfidf": i,
#             "lexrank": j,
#             "textrank": k
#         }
        
#         get_rouge_score(weight)

In [11]:
weight = {
    "tfidf": 0.0,
    "lexrank": 0.8,
    "textrank": 0.2
}

get_rouge_score_and_saved(weight)

Test on training set
Test on valid set


1it [00:00,  1.61it/s]

Predict:
 Ngày 22/5 một chuyến bay khởi_hành từ Villahermosa và hướng đến Mexico_City thì một chú chim bất_ngờ lao vào tuabin động_cơ khiến máy_bay phải hạ_cánh khẩn_cấp may_mắn không xảy ra tai_nạn đáng tiếc nào.Theo RT vài phút sau khi chiếc máy_bay Airbus_A320 cất_cánh lúc 22 h ngày 23/8 một tiếng nổ lớn đã vang lên và lửa bắt_đầu phụt ra từ động_cơ bên phải.Phi_hành_đoàn nhận được cảnh_báo khi máy_bay đang ở độ cao tương_đương 4.000 m và đã quay trở_lại hạ_cánh tại sân_bay Guadalajara 45 phút sau khi cất_cánh.Vụ cháy khiến các hành_khách hoảng_loạn khóc_lóc la_hét và cầu_nguyện
Golden:
 Chuyến bay của hãng Viva_Aerobus mang số_hiệu VB518 có sức chứa 186 hành_khách khởi_hành từ Guadalajara Mexico tối 24/8 giờ_địa_phương dự_kiến kéo_dài 3 tiếng.Khoảng 10 phút sau khi cất_cánh hành_khách phát_hiện tia lửa bắn ra từ động_cơ bên phải của máy_bay.45 phút sau khi cất_cánh khi máy_bay đang ở độ cao tương_đương 4.000 m phi_hành_đoàn đã cho máy_bay quay trở_lại hạ_cánh tại sân_bay Guadalajar

100it [04:15,  2.56s/it]


Predict on test set


1it [00:01,  1.06s/it]

Predict:
 Những ngôi mộ cổ được tìm thấy bởi công ty khảo cổ ArchaeoTask GmbH, trong quá trình khảo sát một mảnh đất gần bờ sông Danube để đào một chiếc ao lớn chứa nước mưa. Với niên đại và giá trị lịch sử đó, các phát hiện tại Geisingen-Gutmadingen - bao gồm một kho hiện vật cực kỳ phong phú - là một kho tàng khảo cổ vĩ đại. Khi địa điểm này được mở từng cái một, người ta phát hiện ra đây là địa điểm Lajia đã biến mất trong trận động đất 4000 năm trước. Cạnh đó là 140 ngôi mộ đầu thời Trung Cổ, có niên đại từ năm 500 đến 600 sau Công Nguyên, chứa hàng hóa bao gồm kiếm, thương, khiên, lược xương, ly uống nước và hoa tai. Những ngôi mộ thời Trung Cổ thì thuộc về thời kỳ mà lãnh chúa Đức Odoacer phế truất hoàng đế La Mã Romulus Augustus, một phần của "Thời kỳ di cư" cực kỳ quan trọng ở châu Âu, đánh dấu sự di chuyển của các bộ tộc, cách họ chinh phục lẫn nhau, đẩy nhau sang các lãnh thổ mới... từ đó dần định hình nên châu Âu hiện đại. Tại một địa điểm thuộc quận Geisingen-Gutmadingen củ

300it [15:11,  3.04s/it]

Start write to txt
Done write to txt
Using weight
 {'tfidf': 0.0, 'lexrank': 0.8, 'textrank': 0.2}
Valid result
         name      mean       min       max       std
0  rouge_1_p  0.414330  0.196970  0.777778  0.136481
1  rouge_1_r  0.611160  0.200000  0.986667  0.129249
2  rouge_1_f  0.482777  0.267123  0.865497  0.120605
3  rouge_2_p  0.238921  0.035556  0.736842  0.149251
4  rouge_2_r  0.385006  0.041667  0.947368  0.170378
5  rouge_2_f  0.286513  0.046110  0.743802  0.153401
6  rouge_l_p  0.390106  0.171717  0.777778  0.142303
7  rouge_l_r  0.574309  0.127273  0.986667  0.138422
8  rouge_l_f  0.454431  0.172840  0.865497  0.130666
